In [1]:
import pandas as pd
from geopy.distance import geodesic
from sklearn.neighbors import BallTree
import numpy as np

ModuleNotFoundError: No module named 'geopy'

In [2]:
!pip install geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ----------------------------------- ---- 112.6/125.4 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------  122.9/125.4 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------  122.9/125.4 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------  122.9/125.4 kB 1.8 MB/s eta 0:00:01
   -------------------------------------- 125.4/125.4 kB 526.0 kB/s eta 0:00:00
   ---------------------------------------- 

In [ ]:
import pandas as pd
from geopy.distance import geodesic
from sklearn.neighbors import BallTree
import numpy as np

# Load the CSV files
camelsde_df = pd.read_csv('attributes_other_camelsde.csv')
grdc_df = pd.read_csv('attributes_other_grdc.csv')

# Create a BallTree for efficient spatial search
camelsde_coords = np.radians(camelsde_df[['gauge_lat', 'gauge_lon']])
grdc_coords = np.radians(grdc_df[['gauge_lat', 'gauge_lon']])

tree = BallTree(camelsde_coords, metric='haversine')

# Query the GRDC coordinates within a 10 km radius (convert km to radians)
radius = 10 / 6371  # Earth's radius in km
indices = tree.query_radius(grdc_coords, r=radius)

# Collect matching catchments
matches = []
for grdc_idx, camelsde_idxs in enumerate(indices):
    grdc_row = grdc_df.iloc[grdc_idx]
    for camelsde_idx in camelsde_idxs:
        camelsde_row = camelsde_df.iloc[camelsde_idx]
        distance = geodesic(
            (grdc_row['gauge_lat'], grdc_row['gauge_lon']),
            (camelsde_row['gauge_lat'], camelsde_row['gauge_lon'])
        ).km
        
        matches.append({
            'grdc_gauge_name': grdc_row['gauge_name'],
            'grdc_lat': grdc_row['gauge_lat'],
            'grdc_lon': grdc_row['gauge_lon'],
            'camelsde_gauge_name': camelsde_row['gauge_name'],
            'camelsde_lat': camelsde_row['gauge_lat'],
            'camelsde_lon': camelsde_row['gauge_lon'],
            'distance_km': round(distance, 2)
        })

# Convert to DataFrame and save to CSV
similar_catchments = pd.DataFrame(matches)
similar_catchments.to_csv('similar_catchments_10km.csv', index=False)

print("The file 'similar_catchments_10km.csv' has been saved.")


In [5]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np

In [7]:
# Load the two uploaded files
file_camelsgb = r"S:\ashish\01_Analysis\01_Europe_Flood_Analysis\Caravan\Caravan\attributes\camelsgb\attributes_other_camelsgb.csv"
file_grdc = r"S:\ashish\01_Analysis\01_Europe_Flood_Analysis\Caravan\Caravan\attributes\grdc\attributes_other_grdc.csv"

In [8]:
camelsgb_data = pd.read_csv(file_camelsgb)
grdc_data = pd.read_csv(file_grdc)

# Check the first few rows to confirm structure and find the 'area' column
camelsgb_data.head(), grdc_data.head()

(         gauge_id               gauge_name   country  gauge_lat  gauge_lon  \
 0  camelsgb_10002        Ugie at Inverugie  Scotland  57.526691  -1.832792   
 1  camelsgb_10003           Ythan at Ellon  Scotland  57.363771  -2.090017   
 2   camelsgb_1001          Wick at Tarroul  Scotland  58.476196  -3.267061   
 3  camelsgb_11001          Don at Parkhill  Scotland  57.218321  -2.187586   
 4  camelsgb_11003  Don at Bridge of Alford  Scotland  57.242303  -2.717123   
 
           area  
 0   324.312351  
 1   529.992466  
 2   157.445096  
 3  1263.679903  
 4   507.721760  ,
        gauge_id  gauge_lat  gauge_lon                             gauge_name  \
 0  GRDC_1159100   -28.7563    17.7188    ORANGE RIVER, VIOOLSDRIF (27811003)   
 1  GRDC_1159103   -28.9604    19.1521            ORANGE RIVER, PELLA MISSION   
 2  GRDC_1159105   -28.0771    16.8979           ORANGE RIVER, SENDELINGSDRIF   
 3  GRDC_1159110   -31.8030    20.3560  VISRIVIER-OOS, HARDEHEUWEL (27814003)   
 4  GRDC_1

In [9]:
import numpy as np

# Combine the area columns from both datasets
combined_area = pd.concat([camelsgb_data['area'], grdc_data['area']], axis=0)

In [10]:
# Define logarithmic bins
log_bins = [10**i for i in range(1, 8)]  # Bins: 10, 100, 1000, ..., 10,000,000

# Calculate frequencies for logarithmic bins
log_histogram, log_bin_edges = np.histogram(combined_area, bins=log_bins)

# Prepare a table for the log-bin distribution
log_distribution_table = pd.DataFrame({
    "Log Bin Range": [f"{int(log_bin_edges[i])} - {int(log_bin_edges[i+1])}" for i in range(len(log_bin_edges)-1)],
    "Frequency": log_histogram
})

log_distribution_table

,Log Bin Range,Frequency
0,10 - 100,491
1,100 - 1000,1963
2,1000 - 10000,1967
3,10000 - 100000,1044
4,100000 - 1000000,226
5,1000000 - 10000000,30


In [12]:
pip install matplotlib.pyplot 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# Create histogram bins of size 10
# Filter data for areas between 0 and 100
filtered_area = combined_area[combined_area <= 100]
bins = np.arange(0, 110, 10)  # Bins: 0-10, 10-20, ..., 90-100
histogram, bin_edges = np.histogram(filtered_area, bins=bins)

# Create a histogram table
histogram_table = pd.DataFrame({
    "Bin Range": [f"{bin_edges[i]} - {bin_edges[i+1]}" for i in range(len(bin_edges)-1)],
    "Frequency": histogram
})

print(histogram_table)

  Bin Range  Frequency
0    0 - 10         44
1   10 - 20         83
2   20 - 30         75
3   30 - 40         52
4   40 - 50         49
5   50 - 60         59
6   60 - 70         34
7   70 - 80         50
8   80 - 90         49
9  90 - 100         40
